<div style="text-align: center; line-height: 0; padding-top: 9px;">
  <img src="https://blog.scholarnest.com/wp-content/uploads/2023/03/scholarnest-academy-scaled.jpg" alt="ScholarNest Academy" style="width: 1400px">
</div>

#####Cleanup previous runs

In [0]:
%run ../utils/cleanup

#####Setup

In [0]:
%sql
CREATE CATALOG IF NOT EXISTS dev;
CREATE DATABASE IF NOT EXISTS dev.demo_db;

CREATE OR REPLACE TABLE dev.demo_db.people(
  id INT,
  firstName STRING,
  lastName STRING,
  birthDate STRING
) USING DELTA;

#####1. Load data into the delta table

In [0]:
%sql
INSERT OVERWRITE TABLE dev.demo_db.people
SELECT id, fname as firstName, lname as lastName, dob as birthDate
FROM JSON.`/mnt/files/dataset_ch7/people.json`

#####2. Delete one record from the delta table

In [0]:
%sql
delete from dev.demo_db.people where firstName = "M David"

#####3. Update one record in delta table

In [0]:
%sql
update dev.demo_db.people 
set firstName = initCap(firstName), 
    lastName = initCap(lastName)
where birthDate = '1975-05-25'

#####4. Execute a merge statement into a delta table

In [0]:
%sql
merge into dev.demo_db.people tgt
using (select id, fname as firstName, lname as lastName, dob as birthDate
       from json.`/mnt/files/dataset_ch7/people.json`) src
on tgt.id = src.id
when matched and tgt.firstName = 'Kailash' then
  delete
when matched then
  update set tgt.birthDate = src.birthDate
when not matched then
  insert *


#####5. Show delta table version history

In [0]:
%sql
describe history dev.demo_db.people

#####6. Show the most recent version of the data from delta table

In [0]:
%sql
select * from dev.demo_db.people

#####7. Show version 1 of the data from the delta table

In [0]:
%sql
select * from dev.demo_db.people version as of 1

#####8. Show data from the delta table at a given timestamp

In [0]:
%sql
select * from dev.demo_db.people timestamp as of '2023-12-16T05:12:50Z'

#####9. Delete the delta table data by mistake

In [0]:
%sql
delete from dev.demo_db.people

#####10. Rollback your delete and restore the table to a privious version

In [0]:
%sql
describe history dev.demo_db.people

In [0]:
%sql
restore table dev.demo_db.people to timestamp as of '2023-12-16T05:14:39Z'

#####11. Read version 1 of the delta table using Dataframe API

In [0]:
%python
spark.read.option("versionAsOf", "1").table("dev.demo_db.people").display()

#####12. Read the delta table version at a given timestamp using Dataframe API

In [0]:
%python
spark.read.option("timestampAsOf", "2023-12-16T05:14:40Z").table("dev.demo_db.people").display()

#####13. Restore the delta table to version 1 using the API

In [0]:
%python
from delta import DeltaTable
people_dt = DeltaTable.forName(spark, "dev.demo_db.people")
people_dt.restoreToVersion(1)

In [0]:
%sql
select * from dev.demo_db.people

&copy; 2021-2023 ScholarNest Technologies Pvt. Ltd. All rights reserved.<br/>
Apache, Apache Spark, Spark and the Spark logo are trademarks of the <a href="https://www.apache.org/">Apache Software Foundation</a>.<br/>
Databricks, Databricks Cloud and the Databricks logo are trademarks of the <a href="https://www.databricks.com/">Databricks Inc</a>.<br/>
<br/>
<a href="https://www.scholarnest.com/privacy/">Privacy Policy</a> | 
<a href="https://www.scholarnest.com/terms/">Terms of Use</a> | <a href="https://www.scholarnest.com/contact/">Contact Us</a>